In [1]:
! pip install -U tfx

Requirement already up-to-date: tfx in /usr/local/lib/python3.6/dist-packages (0.21.2)


In [2]:
%tensorflow_version 2.x
import pprint
import tensorflow as tf

from pathlib import Path
from tfx.components import CsvExampleGen
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import  InteractiveContext

from tfx.utils.dsl_utils import external_input
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tensorflow_transform.tf_metadata import schema_utils
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip
context = InteractiveContext()
pp = pprint.PrettyPrinter()

TensorFlow 2.x selected.


/usr/local/lib/python3.6/dist-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [3]:
! mkdir data
! cp ./sample_data/california*csv  data/

mkdir: cannot create directory ‘data’: File exists


In [0]:
data_path = Path('./data')

In [5]:
example_gen = CsvExampleGen(input=external_input(data_path.as_posix()))
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-03-15T13_05_17.651785-lys89ds9/CsvExampleGen/examples/1, id: 2)]
        ))

In [6]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen) 

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-03-15T13_05_17.651785-lys89ds9/StatisticsGen/statistics/2, id: 3)]
        ))

In [7]:
# Generates schema based on statistics files.
infer_schema = SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(infer_schema)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-03-15T13_05_17.651785-lys89ds9/SchemaGen/schema/3, id: 4)]
        ))

In [8]:
%%skip_for_export

artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2020-03-15T13_05_17.651785-lys89ds9/CsvExampleGen/examples/1
This cell will be skipped during export to pipeline.


In [9]:
infer_schema.outputs['schema'].get()[0].uri

'/tmp/tfx-interactive-2020-03-15T13_05_17.651785-lys89ds9/SchemaGen/schema/3'

In [0]:
from tensorflow_data_validation import load_schema_text
import pyarrow as pa
import pandas as pd

In [0]:
def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

In [0]:
train_uri  = Path(example_gen.outputs['examples'].get()[0].uri) / 'train'
feature_spec = schema_utils.schema_as_feature_spec(
  load_schema_text(list(Path(infer_schema.outputs['schema'].get()[0].uri).iterdir())[0].as_posix())
).feature_spec
tfrecord_filenames = [name.as_posix() for name in train_uri.iterdir()]
dataset=tf.data.experimental.make_batched_features_dataset(
    tfrecord_filenames, batch_size=256, reader=_gzip_reader_fn, features=feature_spec)

In [0]:
record = [pa.RecordBatch.from_pandas(
    pd.DataFrame.from_dict(data={k.strip('"'):v.numpy()[0]
     for k, v in data.items()}))
  for data in dataset.take(1)
]
table = pa.Table.from_batches(record)

In [14]:
table.to_pandas()

,households,housing_median_age,latitude,longitude,median_house_value,median_income,population,total_bedrooms,total_rooms
0,438.0,40.0,34.150002,-118.110001,196100.0,2.6172,1038.0,509.0,1950.0


# Custom Sklearn API Components

In [0]:
from tfx.components.base import base_component
from tfx.components.base import executor_spec

from tfx.types import standard_artifacts
from tfx.types.component_spec import ChannelParameter
from tfx.types.component_spec import ComponentSpec
from tfx.types.component_spec import ExecutionParameter

from typing import Optional, Text, NamedTuple
from sklearn.base import BaseEstimator

In [0]:
SKTrainerResult = NamedTuple('SKTrainerResult', 
                           [('model', BaseEstimator)])

In [0]:
LABEL_KEY = 'median_house_value'

In [0]:
import json
import os
from typing import Any, Dict, List, Text
import absl
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx import types
from tfx.components.base import base_executor
from tfx.components.base import executor_spec
from tfx.components.base import base_component
from tfx.types import artifact_utils
from tfx.utils import import_utils
from tfx.utils import io_utils
import tensorflow_datasets as tfds
import joblib

In [38]:
%%writefile build_model.py
def build_lgbm_reg(**kwargs):
  from lightgbm import LGBMRegressor
  return LGBMRegressor(**kwargs)



Overwriting build_model.py


In [0]:
class SKTrainerExecustor(base_executor.BaseExecutor):
  def _gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  
  def _flatten(self, examples):
    return dict(**{k.strip('"'):tf.reshape(v, [-1]) for k, v in examples.items()})

  def Do(self, input_dict: Dict[Text,List[types.Artifact]],
         output_dict: Dict[Text, List[types.Artifact]],
         exec_properties: Dict[Text, Any]) -> None:
    train_path = artifact_utils.get_split_uri(input_dict['examples'], 'train')
    tfrecord_filenames = [name.as_posix() for name in train_uri.iterdir()]
    schema_file = io_utils.get_only_uri_in_dir(
        artifact_utils.get_single_uri(input_dict['schema']))
    schema = io_utils.parse_pbtxt_file(schema_file, schema_pb2.Schema())
    feature_spec = schema_utils.schema_as_feature_spec(
      load_schema_text(list(Path(infer_schema.outputs['schema'].get()[0].uri).iterdir())[0].as_posix())
    ).feature_spec

    tfrecord_filenames = [name.as_posix() for name in train_uri.iterdir()]
    dataset=tf.data.experimental.make_batched_features_dataset(
    tfrecord_filenames, batch_size=2048, reader=_gzip_reader_fn, features=feature_spec)

    dataset = dataset.map(self._flatten).prefetch(tf.data.experimental.AUTOTUNE)
    dataset = tfds.as_numpy(dataset)

    X = pd.concat([pd.DataFrame.from_dict(data={k.strip('"'):v.tolist() for k, v in data.items()})
          for data in dataset])
    
    model_fn = import_utils.import_func_from_source(
        exec_properties.get('sklearn_model'), 'build_lgbm_reg'
    )
    model = model_fn(**{'boosting':'gbdt', 'n_estimators':500})
    y = X.pop(LABEL_KEY)
    model.fit(X, y)
    saved_path = Path(artifact_utils.get_single_uri(output_dict['model_export_path']))
    saved_path /= 'model.pkl'
    joblib.dump(model, saved_path)

In [0]:
class SKTrainerSpec(ComponentSpec):
  PARAMETERS = {
      'hyperparameters': ExecutionParameter(type=standard_artifacts.HyperParameters, optional=True),
      'sklearn_model': ExecutionParameter(type=(str, Text))
  }
  INPUTS = {
      'examples': ChannelParameter(type=standard_artifacts.Examples),
      'schema': ChannelParameter(type=standard_artifacts.Schema)
  }
  OUTPUTS = {
     'model_export_path' : ChannelParameter(type=standard_artifacts.Model)
  }
  _OUTPUT_COMPATIBILITY_ALIASES = {
      'model': 'model_export_path'
  }

In [0]:
class SKTrainer(base_component.BaseComponent):
  SPEC_CLASS = SKTrainerSpec
  EXECUTOR_SPEC = executor_spec.ExecutorClassSpec(SKTrainerExecustor)

  def __init__(self, 
               examples: types.Channel = None,
               schema: types.Channel = None,
               hyperparameters: types.Channel = None,
               sklearn_model: types.Channel = None,
               model: Optional[types.Channel] = None
               ):
    model = model or types.Channel(
        type=standard_artifacts.Model, artifacts=[standard_artifacts.Model()])
    spec = SKTrainerSpec(examples=examples, schema=schema,
                         hyperparameters=hyperparameters, sklearn_model=sklearn_model,
                         model_export_path=model)
    super(SKTrainer, self).__init__(spec=spec)

In [0]:
component = SKTrainer(examples=example_gen.outputs['examples'],
                      schema=infer_schema.outputs['schema'],
                      sklearn_model='build_model.py',
                      )

In [0]:
context.run(component)

In [0]:
component.outputs['model_export_path'].get()[0].uri, component.outputs['model_export_path'].get()[0].name

In [0]:
saved = Path(component.outputs['model_export_path'].get()[0].uri)/'model.pkl'

In [0]:
model = joblib.load(saved)

In [0]:
X = pd.read_csv('california_housing_train.csv')
y = X.pop('median_house_value')

In [0]:
prediction = model.predict(X)